In [3]:
import numpy as np
from faker import Faker
from freamon import random_subset, load_data, integrate_data, compute_feature_and_label_data, define_model
from arguseyes import ArgusEyes
import logging
from arguseyes.refinements import InputUsage, FairnessMetrics
import time

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

In [4]:
def execute_with_arguseyes(seed):
    
    cmd_args=[str(seed)]

    eq_opp = 0
    rating_usage = None
    product_usage = None
    
    overall_duration = 0
    usage_duration = 0
    fairness_duration = 0

    eyes = ArgusEyes('freamon-testing', './mlruns')
    
    overall_start = time.time()
    with eyes.classification_pipeline_from_py_file('freamon.py', cmd_args=cmd_args) as pipeline:

        usage_start = time.time()
        tagged_inputs = pipeline.compute(InputUsage())
        usage_duration = time.time() - usage_start
        
        fairness_start = time.time()
        priv_tn, priv_fp, priv_fn, priv_tp, dis_tn, dis_fp, dis_fn, dis_tp = \
            pipeline.compute(FairnessMetrics('third_party', 'Y'))
        fairness_duration = time.time() - fairness_start
        
        eq_opp = (float(dis_fn) / (dis_fn + dis_tp)) - (float(priv_fn) / (priv_fn + priv_tp))
        #print('Equal opportunity', eq_opp)

        for tagged_input in tagged_inputs:
            column_names = list(tagged_input.data.columns)

            if 'star_rating' in column_names:
                rating_usage = np.array(tagged_input.data['__arguseyes__is_used'])
                #print('# ratings used', np.sum(rating_usage))

            if 'product_title' in column_names:
                product_usage = np.array(tagged_input.data['__arguseyes__is_used'])
                #print('# products used', np.sum(product_usage))  

    overall_duration = time.time() - overall_start                

    return rating_usage, product_usage, eq_opp, overall_duration, usage_duration, fairness_duration            

In [5]:
def execute_manually(seed):
    
    fake = Faker()
    fake.seed_instance(seed)
    np.random.seed(seed)
    
    numerical_columns = random_subset(['total_votes', 'star_rating'])
    categorical_columns = random_subset(['customer_id', 'product_id', 'vine', 'category'])
    final_columns = list(set(numerical_columns + categorical_columns + \
      ['text', 'is_helpful', 'review_date'] + ['third_party', 'product_id', 'rating_id']))

    reviews, ratings, products, categories = load_data()
    
    ratings['rating_id'] = range(len(ratings))
    
    integrated_data = integrate_data(reviews, ratings, products, categories, fake)
    train_data, train_labels, test_data, test_labels = \
        compute_feature_and_label_data(integrated_data, final_columns, fake)
    
    sklearn_model = define_model(numerical_columns, categorical_columns)
    
    model = sklearn_model.fit(train_data, train_labels)

    #print('Train accuracy', model.score(train_data, train_labels))
    #print('Test accuracy', model.score(test_data, test_labels))
    
    # Group fairness computation
    from sklearn.metrics import confusion_matrix

    eval_data = test_data.copy(deep=True)

    eval_data['true'] = test_labels
    eval_data['pred'] = model.predict(test_data)
    
    priv = eval_data[eval_data.third_party=='Y']
    dis = eval_data[eval_data.third_party=='N']

    priv_tn, priv_fp, priv_fn, priv_tp = confusion_matrix(priv['true'], priv['pred']).ravel()
    dis_tn, dis_fp, dis_fn, dis_tp = confusion_matrix(dis['true'], dis['pred']).ravel()

    eq_opp = (float(dis_fn) / (dis_fn + dis_tp)) - (float(priv_fn) / (priv_fn + priv_tp))

    #print('Equal opportunity', eq_opp)
    
    # Data usage
    train_rating_ids = list(set(train_data.rating_id))
    train_product_ids = list(set(train_data.product_id))

    rating_usage = np.array(ratings.rating_id.isin(train_rating_ids).values)
    product_usage = np.array(products.product_id.isin(train_product_ids).values)
    
    #print('# ratings used', np.sum(rating_usage))
    #print('# products used', np.sum(product_usage))
    
    return rating_usage, product_usage, eq_opp

In [9]:
seeds = [1234, 830848, 987496, 429214, 178132, 590519, 725202, 476388, 133576, 515078, 
         720668, 449660, 168556, 74155, 164680, 142209, 181459, 994015, 172466, 529630]

overall_durations = []
usage_durations = []
fairness_durations = []
manual_durations = []

with open("random-testing-log.txt", 'w') as logfile:
    for seed in seeds:
        
        manual_start = time.time()
        manual_rating_usage, manual_product_usage, manual_eq_opp = execute_manually(seed)        
        manual_duration = time.time() - manual_start
        
        arg_rating_usage, arg_product_usage, arg_eq_opp, overall_duration, usage_duration, fairness_duration = \
            execute_with_arguseyes(seed)

        print(overall_duration, usage_duration, fairness_duration, manual_duration)
        
        overall_durations.append(overall_duration)
        usage_durations.append(usage_duration)
        fairness_durations.append(fairness_duration)
        manual_durations.append(manual_duration)
        
        
        #print(f'MANUAL, seed={seed}, # ratings used {np.sum(manual_rating_usage)}, ' +
        #      f'# products used {np.sum(manual_product_usage)}, eq_opp {manual_eq_opp}')

        logfile.write(f'MANUAL, seed={seed}, # ratings used {np.sum(manual_rating_usage)}, ' +
              f'# products used {np.sum(manual_product_usage)}, eq_opp {manual_eq_opp}\n')

        #print(f'AUTO, seed={seed}, # ratings used {np.sum(arg_rating_usage)}, ' +
        #      f'# products used {np.sum(arg_product_usage)}, eq_opp {arg_eq_opp}')

        logfile.write(f'AUTO, seed={seed}, # ratings used {np.sum(arg_rating_usage)}, ' +
              f'# products used {np.sum(arg_product_usage)}, eq_opp {arg_eq_opp}\n')

        ratings_match = np.array_equal(manual_rating_usage, arg_rating_usage)
        products_match = np.array_equal(manual_product_usage, arg_product_usage)
        eq_opps_match = manual_eq_opp == arg_eq_opp

        #print(f'COMPARISON: {ratings_match}, {products_match}, {eq_opps_match}')       
        logfile.write(f'COMPARISON: {ratings_match}, {products_match}, {eq_opps_match}\n')

        print('---------------------------------------\n')              


print(np.mean(overall_durations), np.std(overall_durations))
print(np.mean(usage_durations), np.std(usage_durations))
print(np.mean(fairness_durations), np.std(fairness_durations))
print(np.mean(manual_durations), np.std(manual_durations))
        

INFO:Patching sys.argv with ['eyes', '1234']
INFO:Created run 127d25a0a91346c9ab1c9f55eac5e928 for this invocation
INFO:Executing instrumented user pipeline with mlinspect
INFO:Redirecting the pipeline's stdout to arguseyes-pipeline-output.txt
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/mlinspect/monkeypatching/_monkey_patching_utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = original_func(*args, **kwargs)
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

68.77027606964111 22.80372977256775 13.78226089477539 6.545558929443359
---------------------------------------



INFO:Patching sys.argv with ['eyes', '830848']
INFO:Created run 90f86b56c19646cb9273daadae1d258d for this invocation
INFO:Executing instrumented user pipeline with mlinspect
INFO:Redirecting the pipeline's stdout to arguseyes-pipeline-output.txt
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/mlinspect/monkeypatching/_monkey_patching_utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = original_func(*args, **kwargs)
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

74.38464164733887 22.501452922821045 13.79028606414795 12.205785989761353
---------------------------------------



INFO:Patching sys.argv with ['eyes', '987496']
INFO:Created run 0410624518164b36bb0855878d95456c for this invocation
INFO:Executing instrumented user pipeline with mlinspect
INFO:Redirecting the pipeline's stdout to arguseyes-pipeline-output.txt
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/mlinspect/monkeypatching/_monkey_patching_utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = original_func(*args, **kwargs)
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

...

74.11581587791443 23.045017957687378 13.572591066360474 10.630373001098633
---------------------------------------



INFO:Patching sys.argv with ['eyes', '172466']
INFO:Created run f6fcab457517496fb7511efa45a6fa7b for this invocation
INFO:Executing instrumented user pipeline with mlinspect
INFO:Redirecting the pipeline's stdout to arguseyes-pipeline-output.txt
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/mlinspect/monkeypatching/_monkey_patching_utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = original_func(*args, **kwargs)
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

110.09132194519043 22.772450923919678 13.67187786102295 11.272171020507812
---------------------------------------



INFO:Patching sys.argv with ['eyes', '529630']
INFO:Created run 51b43f84a395427792986f97496798df for this invocation
INFO:Executing instrumented user pipeline with mlinspect
INFO:Redirecting the pipeline's stdout to arguseyes-pipeline-output.txt
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/mlinspect/monkeypatching/_monkey_patching_utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = original_func(*args, **kwargs)
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

90.49323678016663 22.544352769851685 13.58112907409668 10.070698976516724
---------------------------------------

286.2640322804451 525.3708072983862
22.90760885477066 0.5934961703715063
13.686275196075439 0.30251135351777364
8.298914170265197 3.5168222008244556
